In [1]:
import pandas as pd


In [5]:
val = pd.read_csv("../data/toronto_valentine_20190210_20190216.csv")
hoc = pd.read_csv("../data/korea_toronto_hocky_20190410_20190424.csv")


In [17]:
hoc[hoc.bar_id == 19088].order_id.value_counts() 

521095549    375
513690152     60
513558466     60
521765889     55
519022853     47
513558528     41
519021838     36
521911713     36
521943786     34
513690401     32
522778513     31
525659494     30
524542006     30
523722790     30
524459282     30
525659502     29
522779177     29
519022596     28
524221650     28
523539432     27
521609487     24
521068167     24
522779124     24
512704015     23
513462991     23
513690475     23
520253648     22
512806163     22
514690232     22
525623573     22
            ... 
524442311      1
512806168      1
525522783      1
512570122      1
522554472      1
513335779      1
512742411      1
523336710      1
519023098      1
513325747      1
519019834      1
519632431      1
519023097      1
512855584      1
521588724      1
512704022      1
514472272      1
524853696      1
513335776      1
522599902      1
522633575      1
521096512      1
514472296      1
514492365      1
521095550      1
511233432      1
511233424      1
522697104     

In [157]:
data = hoc[~(hoc.guest_count == 0)]

In [ ]:
Lets remove those that have a guest count of zeros

In [58]:
len(data)

478447

In [34]:
data.guest_count.value_counts(normalize=True) #cool

1      0.369362
2      0.281296
3      0.126547
4      0.086572
5      0.042671
6      0.027123
7      0.014917
8      0.010829
9      0.007809
10     0.005856
11     0.004642
12     0.004310
13     0.002947
14     0.002381
15     0.001651
19     0.001419
18     0.001323
17     0.001244
16     0.001158
21     0.000765
20     0.000665
23     0.000614
22     0.000500
26     0.000462
48     0.000445
24     0.000431
58     0.000397
27     0.000320
37     0.000265
33     0.000230
35     0.000184
29     0.000163
38     0.000153
25     0.000075
32     0.000069
28     0.000050
34     0.000048
42     0.000036
30     0.000023
52     0.000017
73     0.000017
60     0.000008
55     0.000004
50     0.000002
108    0.000002
Name: guest_count, dtype: float64

In [158]:
data["order_time"] = pd.to_datetime(data["order_time"], format="%Y-%m-%d %H:%M:%S.%f")
data["order_item_time"] = pd.to_datetime(data["order_item_time"], format="%Y-%m-%d %H:%M:%S.%f")
data["order_time_closed"] = pd.to_datetime(data["order_time_closed"], format="%Y-%m-%d %H:%M:%S.%f")

/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [159]:
data = pd.concat([data, pd.get_dummies(data["category_id"], prefix="category_id")], axis=1)
# count total of orders per category id
category_ids = [float(i) for i in range(1, 7)]
for category_id in category_ids:
    data["total_orders_category_id_" + str(category_id)] =\
        data.groupby("order_id")["category_id_" + str(category_id)].transform("sum")

In [160]:
total_categories_ids = ["total_orders_category_id_" + str(float(i)) for i in range(1, 7)]
total_categories_ids_to_sum = [column for column in total_categories_ids if column != "total_orders_category_id_5.0"]
data["total_orders"] = data.apply(lambda order: sum(order[column] for column in total_categories_ids_to_sum), axis=1)


In [161]:
data["order_day_of_week"] = data.order_time.apply(lambda ticket: ticket.day_name())


# 4. Add `period_of_day` feature ('breakfast', 'lunch', 'afternoon', 'dinner', 'night')
data["order_hour"] = data.order_time.apply(lambda ticket: ticket.hour)

def period_of_day(hour):
    if hour >= 6 and hour < 11: return 'breakfast'
    elif hour >= 11 and hour < 14: return 'lunch'
    elif hour >= 14 and hour < 18: return 'afternoon'
    elif hour >= 18 and hour < 21: return 'dinner'
    elif hour >= 21 and hour < 23: return 'hang_out'
    elif hour >= 23 or hour < 6: return 'night'

data["period_of_day"] = data.apply(lambda order: period_of_day(int(order["order_hour"])), axis=1)
#data.drop("order_hour", axis=1, inplace=True)

# 5. Add `is_weekend` feature
weekend = ["Friday", "Saturday", "Sunday"]
data["is_weekend"] = data.order_day_of_week.apply(lambda ticket_day: ticket_day in weekend)

In [162]:
sauces = set()
sub = set()
# drop unrelevant titles
def filter_titles(row):
    title = row["title"].lower()
    price = row["sales_before_tax"]

    if "xtra" in title:
        return False
    if "sub" in title and price == 0:
        sub.add(title)
        return False
    elif "add" in title:
        return False
    elif "sauce" in title:
        for t in foods_w_sauce: 
            if t in title: return True
        for t in remove_w_sauce:
            if t in title: return False
        sauces.add(title)
        return False
    elif "no " in title:
        return False
    elif "no." in title:
        return False
    elif "-no " in title:
        return False
    elif "side " in title:
        return False
    elif "+" in title:
        return False
    elif "dip" in title:
        return False
    elif "blue cheese" in title:
        return False
    elif "bbq" in title:
        return False
    elif "n/c" in title:
        return False
    elif "s/o" in title:
        return False
    elif title == '' or title == 'garlic.aioli' or title == 'gluten' or title == 'hot n honey' or title == 'honey garlic' or title == 'kids.' or title == 'to go':
        return False
    else:
        return True

foods_w_sauce = ["fingers", "spaghetti", "poutine", "wings", "pate", "bowl", "fries", "rigatoni", "pasta",
                "linguini", "frite"]
remove_w_sauce = ["no wing", "for wing", "on", "side"]
whitelist = set('.abcdefghijklmnopqrstuvwxyz \\//')
    
 

In [164]:
data.title = data.title.apply(lambda x: str(x))
data = data[data.apply(lambda x: filter_titles(x), axis=1)]


In [165]:
   

sharable_foods = [
    "pizza", "cake", "hot pot", "nachos", "guac", "wings", "Focaccia", "bread", "fries", "pretzels",
"quesadilla", "nuts", "fondue", "calamari", "fingers sauced","chicken fingers", "chkn fingers", "quesa stack" 
]

def find_sharable(title):
    title = " ".join(title.lower().split("."))
    for s in sharable_foods:
        if s in title:
            return 1
    return 0

data["sharable"] = data.title.apply(lambda x: find_sharable(x))




# meal with kids
data["kids_meal"] = data.title.apply(lambda x: 1 if 'kid' in x else 0)
# birthday
data["birthday"] = data.title.apply(lambda x: 1 if 'birthday' in x else 0)


/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [167]:
data["order_minute"] = data.order_time.apply(lambda ticket: ticket.minute)
data["order_item_minute"] = data.order_item_time.apply(lambda ticket: ticket.minute)
data["order_item_hour"] = data.order_item_time.apply(lambda ticket: ticket.hour)
data["order_close_minute"] = data.order_time_closed.apply(lambda ticket: ticket.minute)
data["order_close_hour"] = data.order_time_closed.apply(lambda ticket: ticket.hour)

In [86]:
data.order_time[1]

Timestamp('2019-04-16 12:01:00')

In [95]:
d = (data.order_time[14] - data.order_time[1])

In [101]:
d.components.minutes

49

In [97]:
d / pd.Timedelta('1 hour')

7.816666666666666

In [168]:
order_ids = list(data.order_id.value_counts().keys())

In [169]:
import math

def find_total_time(data):
    total_dict = {}
    for order in order_ids:
        total_dict[order] = {}
        df_order = data[data.order_id == order]
        
        #Calculating total time
        closing_time = list(df_order.order_time_closed)[0]
        item_times = list(df_order.order_item_time.sort_values())
        earliest_ordering_time = item_times[0]
        order_time = list(df_order.order_time)[0]
        if order_time < earliest_ordering_time: 
            total_time = closing_time - order_time
        else: total_time = closing_time - earliest_ordering_time
        total_dict[order]["total_time"] = total_time
        
        #Calculating mealflow
        prev = 0
        prev_meal_flow_step = 0
        total_dict[order]["meal_step"] = {}
        total_dict[order]["meal_flow"] = {}
        total_steps = 0
        for i,item in enumerate(item_times):
            total_dict[order]["meal_step"][item] = i
            
            if prev == 0:
                flow_step = math.ceil((order_time - item).components.minutes/4)
                total_dict[order]["meal_flow"][item] = flow_step
                prev = item
                continue
            flow_step = math.ceil((item - prev).components.minutes/4)
            prev = item
            total_dict[order]["meal_flow"][item] = flow_step + prev_meal_flow_step
            prev_meal_flow_step = flow_step
            total_steps += flow_step 
        
        
        total_dict[order]["last_step"] = item_times[-1]
        last_flow_step = math.ceil((closing_time - item_times[-1]).components.minutes/4) + prev_meal_flow_step
        total_dict[order]["meal_flow"][closing_time] = last_flow_step
        total_steps += last_flow_step
        total_dict[order]["total_meal_flow_steps"] = total_steps
     
    return total_dict
  
total_dict = find_total_time(data)

In [170]:
data["total_order_time"] = data.order_id.apply(lambda x: total_dict[x]["total_time"])

In [171]:
data["meal_step"] = data.apply(lambda x: total_dict[x.order_id]["meal_step"][x.order_item_time], axis=1)

In [172]:
data["total_order_time"] = data.order_id.apply(lambda x: total_dict[x]["total_meal_flow_steps"])

In [174]:
data["meal_flow_step"] = data.apply(lambda x: total_dict[x.order_id]["meal_flow"][x.order_item_time], axis=1)

In [175]:
data["last_step"] = data.order_id.apply(lambda x: total_dict[x]["last_step"])

In [ ]:
data.to_csv("hoc_features.csv", index=False)